In [1]:
import numpy as np
from skimage import data, io
from skimage.filters import threshold_otsu
import os
import imageio
from matplotlib import pyplot as plt
import pandas as pd
import napari
import tifffile
from scipy import spatial
from sklearn.preprocessing import normalize

In [2]:
from threadpoolctl import threadpool_limits
import ray
# num_cpus=48
# ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

In [3]:
io_directory = '/mnt/ampa_data01/tmurakami/220305_SMA_nuc_middlehuman/vessel_analysis'

In [4]:
asma_downscale = np.load(os.path.join(io_directory,'vessel.npy'))
dim = asma_downscale.ndim
print(asma_downscale.shape)

(450, 819, 546)


### If meninge mask are required, make it using labkit

In [5]:
# mask_meninge = 1 - tifffile.imread(os.path.join(io_directory,'meninge.tif'))

In [6]:
# Prepare mask. This time, use mask prepared with ClearMap.
mask = np.load(os.path.join(io_directory,'binary_final.npy'))#np.load('/scratch2/Share/tmurakami/220121_human_sma_5mm_1st/vessel/binary_final.npy')
# mask = mask * mask_meninge
mask_1d = mask.flatten()

# Extract the vectors and positions using mask.
points_position_array = np.array(np.where(mask)).T
points_position = points_position_array.tolist()

In [7]:
# manually design your guide vector
# To Do: automation of the vector detection.
guide_coordinate1 = np.array([237.,145.,405.])
guide_coordinate2 = np.array([201.,181.,373.])
guide_vector = guide_coordinate2 - guide_coordinate1
guide_vector = guide_vector / np.linalg.norm(guide_vector)

In [9]:
# Prepare skeleton. Use the skeleton prepared with ClearMap.
skeleton = np.load(os.path.join(io_directory,'skeleton.npy'))
# skeleton = skeleton * mask_meninge
skeleton_1d = skeleton.flatten()

# Extract the position of the skeleton for the vector field analysis.
skeleton_position_array = np.array(np.where(skeleton)).T
skeleton_position = skeleton_position_array.tolist()

In [10]:
def align_vector_sign(vectors, guide_vector=None):
    '''
    Align the sign of the vector by refering a guide vector. If the dot product of the vector and the guide vector is negative, the sign of vector is flipped.
    Highly encourage to make a guide vector before align sign.
    '''
    if (guide_vector is None):
        guide_vector = normalize(np.median(vectors,axis=0)[:,np.newaxis],axis=0).ravel()
    aligned_vectors = np.where(
        np.repeat(np.expand_dims(np.matmul(vectors, guide_vector) >= 0, axis=1), guide_vector.size, axis=1),
        vectors,
        -vectors
    )
    return aligned_vectors

In [11]:
# Vector field analysis on skeleton using neighbors.
kdtree = spatial.KDTree(skeleton_position_array)
k = 27 # Number of neighbors.
mean_skeleton_vectors = []

for point, point_position in enumerate(skeleton_position):
    # Extract vectors from k-nearest neighbors.
    d, neighbors = kdtree.query(point_position,k)
    neighbors = neighbors[d!=0]
    vectors_from_neighbors = normalize(skeleton_position_array[neighbors,:]-point_position,axis=1) # Normalize to equalize the weights
    mean_vector = np.mean(align_vector_sign(vectors_from_neighbors,guide_vector),axis=0) # Use arithmetic mean.
    mean_vector = normalize(mean_vector[:,np.newaxis],axis=0).ravel()
    mean_skeleton_vectors.append(mean_vector)
mean_skeleton_vectors = np.array(mean_skeleton_vectors)

In [12]:
# Expansion of vector field to binarized image.
kdtree = spatial.KDTree(skeleton_position_array)
k = 1 # Number of neighbors in skeleton.
point_vectors = []

for point, point_position in enumerate(points_position):
    _, neighbors = kdtree.query(point_position,k)
    neighbor_vector = mean_skeleton_vectors[neighbors]
#     if k > 1:
#         mean_neighbor_vectors =np.mean(neighbor_vector,axis=0)
#         neighbor_vector = normalize(mean_neighbor_vectors[:,np.newaxis],axis=0).ravel()
    point_vectors.append(neighbor_vector)
point_vectors = np.array(point_vectors)

In [ ]:
'''Start denoising'''

In [13]:
@ray.remote
def get_neighbor_vectors(point_position, kdtree, vectors, radius):
    if not isinstance(point_position, np.ndarray):
        point_position = np.array(point_position)
    neighbors = kdtree.query_ball_point(point_position,radius)
    neighbor_vectors = vectors[neighbors,:]
    return neighbor_vectors

@ray.remote
def get_median_vector(vectors):
    # Ideally, the medoid vector should be calculated, but it is resource demanding. Instead, calculate the median in each dimension and normalize to a unit vector.
    """
    vectors: ndarray
    """
    median_vector = normalize(np.median(vectors,axis=0)[:,np.newaxis],axis=0).ravel()
    return median_vector

@ray.remote
def get_point_vector(point, vectors):
    return vectors[point]

@ray.remote
def single_thread_align_vector_sign(vectors, guide_vector):
    '''
    Align the sign of the vector by refering a guide vector. If the dot product of the vector and the guide vector is negative, the sign of vector is flipped.
    '''
    with threadpool_limits(limits=1, user_api='blas'):
        aligned_vectors = np.where(
            np.repeat(np.expand_dims(np.matmul(vectors, guide_vector) >= 0, axis=1), guide_vector.size, axis=1),
            vectors,
            -vectors
        )
    return aligned_vectors

@ray.remote
def select_point_in_dot_product_space(point_vector, neighbor_vectors, median_vector, k=10):
    with threadpool_limits(limits=1, user_api='blas'):
        # Calculate the dot product
        dot_product = np.matmul(neighbor_vectors, median_vector) # This is done in parallel otherwise stated.
        dot_product_of_point = np.matmul(point_vector,median_vector)
    if k>dot_product.size:
        selection = False
    else:
        # Find k neighbors in dot product space
        dot_product_neighbors = dot_product[np.argsort(np.abs(dot_product-dot_product_of_point))][0:k]
        # Calculate the null density if the density is even distribution.
        null_density = dot_product.size*(dot_product_neighbors.max()-dot_product_neighbors.min())
        selection = (k>null_density)
    # Second selection using otsu thresholding
    if selection:
        thresh = threshold_otsu(dot_product)
        selection = (dot_product_of_point>thresh)
    return selection

@ray.remote
def dot_product_vectors(vector1, vector2):
    with threadpool_limits(limits=1, user_api='blas'):
        dot_product = np.matmul(vector1, vector2)
    return dot_product

In [14]:
%%time
kdtree = spatial.KDTree(points_position_array)
radius = 42 # pixel unit. 500 / voxel micrometer works well. diameter in real scale: 2 * radius * voxelsize. 
k = 10 # Number of neighbor in dot product space. Note this is not a number of neighbor in 3D image space.
keeping = []
# dot_p = []

kdtree_id = ray.put(kdtree)
vectors_id = ray.put(point_vectors)

for point, point_position in enumerate(points_position):
    point_vector = get_point_vector.remote(point, vectors_id)
    # Get vectors in neighbor points.
    neighbor_vectors = get_neighbor_vectors.remote(point_position, kdtree_id, vectors_id, radius)
    # Make representitive vector
    median_vector = get_median_vector.remote(neighbor_vectors)
    neighbor_vectors = single_thread_align_vector_sign.remote(neighbor_vectors, median_vector) # Fix the sign of vectors.

    keeping.append(select_point_in_dot_product_space.remote(point_vector, neighbor_vectors, median_vector, k))
    # dot_p.append(dot_product_vectors.remote(point_vector,median_vector))

keeping = ray.get(keeping)
extract_idx = np.where(mask_1d)[0][keeping]

(get_point_vector pid=780695) 
(get_median_vector pid=780559) 
(select_point_in_dot_product_space pid=780622) 
(get_neighbor_vectors pid=780697) 
(get_neighbor_vectors pid=780705) 
(get_point_vector pid=780690) 
(select_point_in_dot_product_space pid=780670) 
(get_median_vector pid=780670) 
(single_thread_align_vector_sign pid=780650) 
(get_median_vector pid=780626) 
(get_point_vector pid=780635) 
(get_point_vector pid=780699) 
(get_neighbor_vectors pid=780686) 
(single_thread_align_vector_sign pid=780610) 
(get_median_vector pid=780560) 
(get_point_vector pid=780610) 
(get_neighbor_vectors pid=780606) 
(get_point_vector pid=780648) 
(get_median_vector pid=780616) 
(get_neighbor_vectors pid=780682) 
(get_neighbor_vectors pid=780665) 
(get_neighbor_vectors pid=780612) 
(get_median_vector pid=780618) 
(get_point_vector pid=780622) 
(get_neighbor_vectors pid=780701) 
(get_point_vector pid=780653) 
(get_point_vector pid=780615) 
(get_median_vector pid=780658) 
(get_median_vector pid=780611

In [15]:
if True: # True to save images and variables for later use.
    vec_img = np.zeros(asma_downscale.shape+(dim,)).astype(np.float32)
    extracted = points_position_array[keeping]
    vec_img[tuple(extracted.T)] = point_vectors[keeping,:]
    
    # export extracted vetors as image
    tifffile.imwrite(os.path.join(io_directory,'local_vector.tif'),
                 np.moveaxis(vec_img,-1,1).astype(np.float32),
                 imagej=True,
                 metadata={'spacing': 12, 'unit': 'um', 'axes': 'ZCYX'})
    
    # save variables as .npy
    np.save(os.path.join(io_directory,'extract_idx.npy'), extract_idx)
    np.save(os.path.join(io_directory,'point_vectors.npy'), point_vectors)
    np.save(os.path.join(io_directory,'keeping.npy'), np.asarray(keeping))
    
    # viewer.add_image(vec_img, contrast_limits=[-1,1], rgb=True, name='local_vector', blending='additive')
    # tes_img = np.zeros_like(asma_downscale).astype(float)
    # extracted = points_position_array[keeping]
    # nonextracted = points_position_array[~np.array(keeping)]
    # tes_img[tuple(extracted.T)] = np.array(dot_p)[np.array(keeping)]
    # imageio.mimwrite(os.path.join(io_directory,'local_dot_extracted02.tif'),tes_img)

In [17]:
ray.shutdown()

In [20]:
''' # If image export of the selected vectors are required.
extract_vectors = np.zeros_like(selected_min_eig_vector)
extract_vectors[np.unravel_index(extract_idx,asma_downscale.shape)] = selected_min_eig_vector[np.unravel_index(extract_idx,asma_downscale.shape)]
nonextract_vectors = np.zeros_like(selected_min_eig_vector)
nonextract_vectors[np.unravel_index(nonextract_idx,asma_downscale.shape)] = selected_min_eig_vector[np.unravel_index(nonextract_idx,asma_downscale.shape)]

# Save masked vector as a tif.
extract_vector_tif = os.path.join(io_directory,'vector_dot_product_selection.tif')
imageio.mimwrite(extract_vector_tif,extract_vectors)
nonextract_vector_tif = os.path.join(io_directory,'vector_dot_product_selection_non.tif')
imageio.mimwrite(nonextract_vector_tif,nonextract_vectors)
'''